This script should be run on a folder containing density heatmaps generated from the ImageJ Density Heatmap analysis tool.

It will create an indivdual csv file for each image, and a summary file for the complete set.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import os
import csv
import sys
from scipy.misc import face
from PIL import Image, ImageDraw
from skimage.morphology import convex_hull_image
from skimage import measure
import pandas as pd

In [10]:
# if necessary update the working directory
os.chdir('C:/Users/Luke/Desktop/TestData/Density_Heatmaps')
dirpath = os.getcwd()
print("current directory is : " + dirpath)
foldername = os.path.basename(dirpath)
print("Directory name is : " + foldername)

current directory is : C:\Users\Luke\Desktop\NeuronMaskTest\Density_Heatmaps
Directory name is : Density_Heatmaps


In [11]:
# list tif files in directory
files = [file_i
         for file_i in os.listdir(dirpath)
         if file_i.endswith('.tif')]

In [12]:
# Create empty arrays for measurements
Counter = []
Filename = []
ConvexHullArea = []
PeakDensity = []
Zeros = []

In [13]:
# Loop over files
for file in range(len(files)):
    #print('processing file '+files[file], end='\r')
    print('processing file '+files[file])
    Counter.append(file)
    Filename.append(files[file])
    
    #import image and set channels
    img = io.imread(files[file])
    #set channels
    Soma = img[:,:,0]
    Density = img[:,:,1]
    Neuron = img[:,:,2]
    Distance = img[:,:,3]
    
    #get convex hull
    chull = convex_hull_image(Neuron)
    all_labels = measure.label(chull)
    #blobs_labels = measure.label(chull, background=0)
    properties = measure.regionprops(all_labels)
    ConvexHullArea.append(properties[0].area)
    
    #create mask
    mask = (chull > 0)
       
    #measure itensity (Density) within mask
    region = Density[mask]
    hist = np.histogram(region.ravel(), bins = 200, range = [0, 200])
    
    HistX = np.array(hist[1][0:-1:]) #had to adjust otherwise was 201 long not 200
    HistY = np.array(hist[0])
    
    Total = sum(hist[0])
    
    #HistPC = 
    HistDF = pd.DataFrame({"Intensity" : HistX,"Count" : HistY})
    HistDF.to_csv(files[file]+".csv", index=False)
    
    #get other interesting values:
    #peak density
    PeakDensity.append(np.max(Density))
    #number of zeros within convexhull
    Zeros.append(hist[0][0])
      
    

processing file s407_01_MaxIP_3.tif
processing file s407_04_MaxIP_1.tif
processing file s427_01_MaxIP_1.tif
processing file s427_05_MaxIP_1.tif
processing file s427_08_MaxIP_1.tif
processing file s429_12_MaxIP_1.tif
processing file s429_14_MaxIP_1.tif
processing file s431_24_MaxIP_1.tif
processing file s431_26_MaxIP_1.tif
processing file s431_29_MaxIP_1.tif
processing file s432_02_MaxIP_1.tif
processing file s432_04_MaxIP_1.tif
processing file s432_08_MaxIP_1.tif
processing file s432_11_MaxIP_1.tif
processing file s432_14_MaxIP_2.tif
processing file s471_07_MaxIP_1.tif
processing file s471_11_MaxIP_1.tif
processing file s471_12_MaxIP_1.tif
processing file s476_14_MaxIP_1.tif
processing file s521_a3c1__cut180.tif
processing file s521_a4c1__cut180.tif
processing file s521_a5c2__cut180.tif
processing file s521_a6c1__cut180.tif


In [16]:
Counter = np.array(Counter)
Filename = np.array(Filename)
ConvexHullArea = np.array(ConvexHullArea)
PeakDensity = np.array(PeakDensity)
Zeros = np.array(Zeros)

In [17]:
#Create summary table
df = pd.DataFrame({"Cell_ID" : Counter,"Filename" : Filename, "Convex_Hull_Area_px" : ConvexHullArea, 
                   "Peak_Density" : PeakDensity, "Total_Zeros" : Zeros})
#save summary table
df.to_csv("Summary.csv", index=False)